#1. cosine similarity & 데이터

+ silmilarity measure 로 cossine 함수를 이용

### (1) measure

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
def cossim_matrix(user_movie_matrix_a, user_movie_matrix_b):
    cossim_values = cosine_similarity(user_movie_matrix_a.values, user_movie_matrix_b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = user_movie_matrix_a.index.values, index=user_movie_matrix_a.index) # index.values 하면 name인 movieId를 사용하지 않게 됨 -> index로 movieId 쓴다는 것

    return cossim_df

### (2) data load

In [14]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
ratings_df = pd.read_csv('https://raw.githubusercontent.com/jisy2718/recommender_system/master/data/movielens_small/ratings.csv')
movies_df = pd.read_csv('https://raw.githubusercontent.com/jisy2718/recommender_system/master/data/movielens_small/movies.csv', index_col='movieId')
links_df = pd.read_csv('https://raw.githubusercontent.com/jisy2718/recommender_system/master/data/movielens_small/links.csv')
tags_df = pd.read_csv('https://raw.githubusercontent.com/jisy2718/recommender_system/master/data/movielens_small/tags.csv')

df_lst = [ratings_df, movies_df, links_df, tags_df]
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import pprint
for df in df_lst:
    print(df.shape)
    pprint.pprint(df.head())


# 데이터 쪼개기
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1)
train_df

(100836, 4)
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
(9742, 2)
                                      title                                       genres
movieId                                                                                 
1                          Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
2                            Jumanji (1995)                   Adventure|Children|Fantasy
3                   Grumpier Old Men (1995)                               Comedy|Romance
4                  Waiting to Exhale (1995)                         Comedy|Drama|Romance
5        Father of the Bride Part II (1995)                                       Comedy
(9742, 3)
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3    

,userId,movieId,rating,timestamp
78834,489,3827,2.0,1333232046
7523,51,1639,5.0,1230929619
66122,425,5349,3.0,1106482551
69250,448,3793,4.0,1019125762
95606,600,3821,1.0,1237762470
...,...,...,...,...
50057,322,4226,3.5,1217676500
98047,606,4973,4.5,1171233963
5192,34,318,4.0,1162050357
77708,483,8644,4.0,1178358519


In [4]:
# pivot ratings into movie features
user_movie_matrix = ratings_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)
 
# convert dataframe of movie features to scipy sparse matrix
sparse_mat = csr_matrix(user_movie_matrix.values)


# item들의 cosine similarity
item_cossim_df = cossim_matrix(user_movie_matrix, user_movie_matrix)
item_cossim_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,128,129,132,135,137,140,141,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,287,288,...,170401,170411,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170939,170945,170957,170993,171011,171023,171251,171495,171631,171695,171701,171749,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172497,172547,172577,172583,172585,172587,172589,172591,172637,172705,172793,172825,172875,172881,172887,172909,173145,173197,173205,173209,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173751,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174551,174681,174727,174737,174815,174909,175197,175199,175293,175303,175387,175397,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175705,175707,175743,175781,176051,176101,176329,176371,176389,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178129,178323,178613,178615,178827,179053,179073,179119,179133,179135,179211,179401,179427,179491,179511,179709,179749,179813,179815,179817,179819,179953,180031,180045,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181315,181413,181659,181719,182293,182297,182299,182639,182715,182727,182731,182749,182793,182823,183011,183197,183199,183227,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184257,184349,184471,184641,184721,184791,184931,184987,184997,185029,185031,185033,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.410562,0.296917,0.035573,0.308762,0.376316,0.277491,0.131629,0.232586,0.395573,0.323976,0.219680,0.151236,0.178144,0.104528,0.292152,0.334693,0.167993,0.383641,0.147039,0.335337,0.226570,0.164626,0.215168,0.324072,0.170520,0.106747,0.124567,0.190233,0.047388,0.178382,0.508593,0.505196,0.289150,0.135167,0.351076,0.041815,0.117506,0.101851,0.101023,0.244010,0.185565,0.151085,0.437659,0.320534,0.068056,0.441003,0.200272,0.048123,0.044553,0.000000,0.122502,0.223773,0.265399,0.042562,0.368101,0.148886,0.121151,0.260964,0.152594,0.115753,0.166432,0.243044,0.100208,0.136012,0.180071,0.155493,0.145334,0.187748,0.068056,0.070029,0.159558,0.030077,0.168748,0.093544,0.051042,0.135692,0.139348,0.113947,0.182804,0.149547,0.130742,0.054434,0.170414,0.362094,0.068056,0.074683,0.096246,0.112646,0.214393,0.125683,0.039292,0.4

In [ ]:
# 각 user의 cosine similarity 계산
user_id = 512
user_ratings_df = ratings_df[ratings_df['userId']==user_id]
user_sim_item_df = item_cossim_df.loc[user_ratings_df['movieId']]
user_sim_item_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,128,129,132,135,137,140,141,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,287,288,...,170401,170411,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170939,170945,170957,170993,171011,171023,171251,171495,171631,171695,171701,171749,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172497,172547,172577,172583,172585,172587,172589,172591,172637,172705,172793,172825,172875,172881,172887,172909,173145,173197,173205,173209,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173751,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174551,174681,174727,174737,174815,174909,175197,175199,175293,175303,175387,175397,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175705,175707,175743,175781,176051,176101,176329,176371,176389,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178129,178323,178613,178615,178827,179053,179073,179119,179133,179135,179211,179401,179427,179491,179511,179709,179749,179813,179815,179817,179819,179953,180031,180045,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181315,181413,181659,181719,182293,182297,182299,182639,182715,182727,182731,182749,182793,182823,183011,183197,183199,183227,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184257,184349,184471,184641,184721,184791,184931,184987,184997,185029,185031,185033,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.410562,1.000000,0.282438,0.106415,0.287795,0.297009,0.228576,0.172498,0.044835,0.417693,0.322252,0.168642,0.194398,0.099373,0.229191,0.239683,0.208254,0.182717,0.497560,0.129955,0.273282,0.244361,0.128695,0.203931,0.159834,0.162491,0.151033,0.102551,0.163939,0.018742,0.244840,0.381811,0.453192,0.180312,0.148042,0.359441,0.000000,0.121620,0.044393,0.143692,0.319895,0.130049,0.239552,0.384158,0.418518,0.107667,0.304593,0.089099,0.000000,0.146842,0.000000,0.108920,0.122203,0.342229,0.089539,0.303093,0.075024,0.031081,0.172647,0.105823,0.000000,0.114174,0.236369,0.041356,0.138751,0.220277,0.021085,0.027491,0.239456,0.000000,0.041546,0.045698,0.000000,0.122217,0.017343,0.0,0.049349,0.179548,0.194650,0.158885,0.136197,0.202581,0.226150,0.050817,0.293216,0.080750,0.049945,0.000000,0.087850,0.199852,0.028405,0.046621,0.368346

# 2. Item base 추천모델 구축

## [1] version 1 : center 해서 한 것
+ Charu C. Aggarwal의 Recommender Systems 책의 2.3 Item-Based recommender 이용
+ $$AdjustedCosine(i,j) = \cfrac{\sum_{u\in U_i \cap U_j} s_{ui} \cdot s_{uj} }{\sqrt{\sum_{u\in U_i \cap U_j} s_{ui}^2 } \cdot \sqrt{\sum_{u\in U_i \cap U_j} s_{uj}^2 } }$$ , where $U_i$ 는 item i를 평가한 user set

+ $Q_t(u)$ : user u가 평가한 item 중, target item t와 가장 similar한 top k개 item set

+ user u의 item t에 관한 예측값 $\hat{r}_{ut}=\cfrac{\sum_{j \in Q_t(u)}AdjustedCosine(j,t) \cdot r_{uj}}{\sum_{j\in Q_t(u)}|AdjustedCosine(j,t)|}$

  + basic concept은 user u의 item t에 대한 rating 예측을 할 때,  user u가 rating한 것의 가중평균을 이용한다는 것

### (1) center해서 similarity 구하기

+ 예측하기

In [5]:
# train_df 전처리

# 1. 각 user의 평균으로 rating을 빼준, mean_centered_rating 생성
## [1] 각 user_id 별로, 평균 구하는 dictionary 생성
userId_meanRating_dict = {}
for user_id, group in train_df.groupby('userId'):
    mean = group['rating'].mean()
    userId_meanRating_dict[user_id] = mean

print(userId_meanRating_dict)

## [2] mean_centered_rating column 생성
def mean_center(data ):
    user_id = data[0]
    rating = data[1]

    mean = userId_meanRating_dict[user_id]
    mean_centered_rating = rating - mean
    return round(mean_centered_rating,4)

train_df['mean_centered_rating'] = train_df[['userId','rating']].apply(mean_center, axis=1)
print(train_df)


# 2. user item matrix 만들기
## [1] pivot ratings into movie features
user_movie_matrix_centered = train_df.pivot(
    index='movieId',
    columns='userId',
    values='mean_centered_rating'
).fillna(0)

print(user_movie_matrix_centered)

## [2] cosine similarity centered
item_cossim_df_centered = cossim_matrix(user_movie_matrix_centered, user_movie_matrix_centered)
print(item_cossim_df_centered)

{1: 4.3841807909604515, 2: 4.0227272727272725, 3: 2.3823529411764706, 4: 3.5681818181818183, 5: 3.6285714285714286, 6: 3.4941634241245136, 7: 3.1141732283464565, 8: 3.5945945945945947, 9: 3.2857142857142856, 10: 3.2242990654205608, 11: 3.8085106382978724, 12: 4.462962962962963, 13: 3.586206896551724, 14: 3.212121212121212, 15: 3.5, 16: 3.759493670886076, 17: 4.202247191011236, 18: 3.713554987212276, 19: 2.618018018018018, 20: 3.6457286432160805, 21: 3.246518105849582, 22: 2.7065217391304346, 23: 3.67, 24: 3.652941176470588, 25: 4.791666666666667, 26: 3.1666666666666665, 27: 3.552380952380952, 28: 3.016304347826087, 29: 4.109375, 30: 4.64, 31: 3.9523809523809526, 32: 3.744186046511628, 33: 3.776, 34: 3.4153846153846152, 35: 4.111111111111111, 36: 2.6979166666666665, 37: 3.933333333333333, 38: 3.152542372881356, 39: 3.8658536585365852, 40: 3.7162162162162162, 41: 3.3035714285714284, 42: 3.616045845272206, 43: 4.574468085106383, 44: 3.380952380952381, 45: 3.892523364485981, 46: 4.05405405

In [ ]:
item_cossim_df_centered

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.010161,0.038406,0.031588,-0.118821,0.012621,0.043491,0.002086,-0.130347,-0.082117,0.005366,-0.132328,-0.096186,0.082796,-0.050043,0.005620,0.006175,0.040182,-0.043745,-0.026476,0.014906,-0.011698,-0.055189,0.074656,0.066212,-0.001405,-0.058547,0.049205,0.004124,0.005819,-0.049132,0.102604,0.156350,0.005902,0.193243,-0.082589,-0.102805,-0.007486,0.001399,0.058486,-0.040073,-0.001425,-0.030132,0.097036,0.038637,0.000000,0.188363,0.004099,0.029577,-0.080888,0.000000,-0.042730,0.031276,-0.046535,-0.004485,0.053637,-0.048684,0.077811,-0.096374,-0.049871,-0.002494,-0.059304,0.105848,-0.009806,0.017686,-0.004533,0.074786,-0.069497,-0.040790,0.040675,-0.043094,-0.053941,-0.062488,0.040041,-0.079182,0.061988,0.100369,-0.036684,0.013361,0.052562,-0.036174,-0.010011,0.099409,-0.161869,0.053274,0.027063,0.041655,-0.014

### (2) 예측하기

+ prediction matrix 생성

In [36]:
# 예측결과를 넣을 df를 생성
# index: userId, columns: total movieId
item_prediction_result_df1 = pd.DataFrame(index= user_movie_matrix_centered.columns, columns=user_movie_matrix_centered.index.values)  # row에 user별로 예측값 넣어주기 위해서, transpose 해서 넣기
item_prediction_result_df1

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [37]:
from tqdm import tqdm
# 2. cur_user의 rating과 , weight 가져오기
check = 0
grouped_rating_df = train_df.groupby('userId')
for user_id, user_ratings_df in tqdm(grouped_rating_df):
    # print()
    # print(user_id)
    # print()
    user_ratings_df = user_ratings_df.sort_values('movieId').set_index('movieId')  # item id로 정렬하고, set_index로 item id 넣어주기 (movieId column은 삭제됨)
    # print(user_ratings_df)

    ## [1] cur_user의 weight 가져오기 : (row : user가 평가한 item, col : 전체 item)
    user_sim_item_df = item_cossim_df_centered.loc[user_ratings_df.index]   # index가 item id인 상황
    # print(user_sim_item_df)

    ## [2] cur_user의 rating 가져오기
    cur_user_rating = user_ratings_df['rating']


    ## [3] cur_user의 전체 item에 대한 weighted_sum
    weighted_sum = cur_user_rating.T.dot(user_sim_item_df)  # (1 x m) dot (m x len(item)) : m은 cur_user가 평가한 item의 개수
    # print(weighted_sum)

    ## [4] weighted_sum을 weight의 절대값 합으로 나눈 것
    # print()
    weight_abs_sum = np.absolute(user_sim_item_df).sum(axis=0)
    pred = weighted_sum/weight_abs_sum
    # print(pred)

    item_prediction_result_df1.loc[user_id] = pred
    # if check == 0:
    #     break
item_prediction_result_df1

100%|██████████| 610/610 [00:38<00:00, 15.68it/s]


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2.810068,0.436728,-0.216415,-1.66893,-1.232884,2.848312,-2.056732,-2.069751,-2.486829,0.821781,1.43227,-3.033698,-0.331276,1.127797,-2.576917,2.463085,-0.489858,1.174118,-1.257954,-2.362543,-0.453958,-0.12864,-2.626325,-0.312731,-0.132681,0.675316,-1.762163,-0.869034,1.307121,-0.931434,-1.424156,1.944469,1.103149,0.603783,3.797665,-0.783805,1.424948,0.66481,-1.768197,1.042502,-1.512209,-1.859562,-1.345286,2.638072,-1.0228,-0.374673,2.321938,1.115912,-1.191921,-2.016923,1.978814,-1.777906,-0.089867,-2.078625,-0.682432,0.583138,-0.470682,-2.04844,-1.765669,-2.149397,0.577344,0.805101,0.869198,-1.705374,-0.632319,2.301987,0.560452,-2.628001,0.041248,-1.191921,-1.90255,-2.070177,0.121969,-1.938857,1.014889,-0.832637,0.129832,-2.973007,-0.716339,-0.685627,-2.037451,-1.967648,2.351129,-2.330266,-0.127392,0.389269,-1.105107,-2

In [38]:
# version 1- 2 : 0~5 사이로, truncate 시키기
# pandas apply 참고 : https://kibua20.tistory.com/194
def make_0_5(rating):
    if rating > 5:
        rating = 5
    elif rating < 0:
        rating = 0
    
    return rating
item_prediction_result_df1_2 = item_prediction_result_df1.applymap(make_0_5)
item_prediction_result_df1_2

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2.810068,0.436728,0.000000,0.00000,0.000000,2.848312,0.0,0.0,0.0,0.821781,1.432270,0.000000,0.000000,1.127797,0.0,2.463085,0.000000,1.174118,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.675316,0.000000,0.000000,1.307121,0.000000,0.0,1.944469,1.103149,0.603783,3.797665,0.000000,1.424948,0.664810,0.000000,1.042502,0.000000,0.000000,0.000000,2.638072,0.000000,0.000000,2.321938,1.115912,0.000000,0.000000,1.978814,0.0,0.000000,0.0,0.000000,0.583138,0.000000,0.0,0.000000,0.0,0.577344,0.805101,0.869198,0.0,0.000000,2.301987,0.560452,0.0,0.041248,0.000000,0.000000,0.0,0.121969,0.0,1.014889,0.000000,0.129832,0.0,0.000000,0.000000,0.000000,0.0,2.351129,0.0,0.000000,0.389269,0.000000,0.0,2.170933,0.0,0.000000,0.000000,0.000000,0.923116,1.806391,0.000000,2.485823,0.972373,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0000

## [2] version 2 : minmax scaling 하고, siliarity 구하기

### (1) minmax scaling하고, similarity 구하기

In [15]:
# train_df 전처리

# 1. user 별로 minmax scaling

## [1] 각 user_id 별로, 평균 구하는 dictionary 생성
userId_minmax_dict = {}
for user_id, group in train_df.groupby('userId'):
    min_val = group['rating'].min()
    max_val = group['rating'].max()
    userId_meanRating_dict[user_id] = [min_val, max_val]

# print(userId_meanRating_dict)

## [2] mean_centered_rating column 생성
def minmax_scaler(data ):
    user_id = data[0]
    rating = data[1]

    min_val, max_val = userId_meanRating_dict[user_id]
    if min_val == max_val: # 모두 값이 같다면,
        return round( min_val/5, 4)

    minmax_rating = (rating - min_val)/(max_val - min_val)
    
    return round(minmax_rating,4)

train_df['minmax_rating'] = train_df[['userId','rating']].apply(minmax_scaler, axis=1)


# 2. user item matrix 만들기
## [1] pivot ratings into movie features
user_movie_matrix_minmax = train_df.pivot(
    index='movieId',
    columns='userId',
    values='minmax_rating'
).fillna(0)

# print(user_movie_matrix_minmax)

## [2] cosine similarity minmax
item_cossim_df_minmax = cossim_matrix(user_movie_matrix_minmax, user_movie_matrix_minmax)
print(item_cossim_df_minmax)



           1         2         3         4         5         6         7         8         9         10        11        12        13        14        15        16        17        18        19        20        21        22        23        24        25        26        27        28        29        30        31        32        34        36        38        39        40        41        42        43        44        45        46        47        48        49        50        52        53        54        55        57        58        60        61        62        63        64        65        66        68        69        70        71        72        73        74        75        76        77        78        79        80        81        82        85        86        87        88        89        92        93        94        95        96        97        99        100       101     102       103       104       105     106       107       108       110       111       112     \
mov

### (2) 예측하기

In [16]:
# 예측결과를 넣을 df를 생성
# index: userId, columns: total movieId
item_prediction_result_df2 = pd.DataFrame(index= user_movie_matrix_minmax.columns, columns=user_movie_matrix_minmax.index.values)  # row에 user별로 예측값 넣어주기 위해서, transpose 해서 넣기
item_prediction_result_df2

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [39]:
from tqdm import tqdm
# 2. cur_user의 rating과 , weight 가져오기
check = 0
grouped_rating_df = train_df.groupby('userId')
for user_id, user_ratings_df in tqdm(grouped_rating_df):

    user_ratings_df = user_ratings_df.sort_values('movieId').set_index('movieId')  # item id로 정렬하고, set_index로 item id 넣어주기 (movieId column은 삭제됨)
    # print(user_ratings_df)

    ## [1] cur_user의 weight 가져오기 : (row : user가 평가한 item, col : 전체 item)
    user_sim_item_df = item_cossim_df_minmax.loc[user_ratings_df.index]   # index가 item id인 상황
    # print(user_sim_item_df)

    ## [2] cur_user의 rating 가져오기
    cur_user_rating = user_ratings_df['rating']


    ## [3] cur_user의 전체 item에 대한 weighted_sum
    weighted_sum = cur_user_rating.T.dot(user_sim_item_df)  # (1 x m) dot (m x len(item)) : m은 cur_user가 평가한 item의 개수
    # print(weighted_sum)


    ## [4] weighted_sum을 weight의 절대값 합으로 나눈 것
    # print()
    weight_abs_sum = np.absolute(user_sim_item_df).sum(axis=0)
    pred = weighted_sum/weight_abs_sum
    # print(pred)

    item_prediction_result_df2.loc[user_id] = pred
    # if check == 0:
    #     break
item_prediction_result_df2

100%|██████████| 610/610 [00:36<00:00, 16.83it/s]


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.407514,4.384467,4.355089,4.470909,4.376105,4.385486,4.319691,4.383474,4.190147,4.33828,4.348829,4.347823,4.450377,4.349715,4.298257,4.390393,4.345212,4.368724,4.379723,4.357074,4.388552,4.357529,4.403647,4.370305,4.374988,4.359944,4.350895,4.37107,4.43818,4.260065,4.350113,4.38997,4.408441,4.378063,4.34535,4.378539,3.587465,4.367231,4.376309,4.373927,4.332462,4.372274,4.356777,4.412054,4.362278,4.398821,4.423778,4.402607,4.301797,4.311252,4.43058,4.387863,4.405284,4.363872,4.329956,4.358035,4.482198,4.186353,4.347063,4.311388,4.342152,4.384387,4.340449,4.388283,4.447681,4.389927,4.380393,4.379494,4.351366,4.301797,4.379757,4.329474,4.313949,4.392697,4.42898,4.388164,4.350749,4.317397,4.366232,4.333555,4.373539,4.304293,4.41289,4.348336,4.425272,4.406855,4.301523,4.363825,4.471181,NaN,3.679558,4.399108,4.418263,4.45332

## [3] version 3 : 그냥 similarity 계산한 것


### (1) similarity 구하기

In [40]:
# 1. user item matrix 만들기
## [1] pivot ratings into movie features
user_movie_matrix = train_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

print(user_movie_matrix)

## [2] cosine similarity
item_cossim_df = cossim_matrix(user_movie_matrix, user_movie_matrix)
print(item_cossim_df)

userId   1    2    3    4    5    6    7    8    9    10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98   99   100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  \

### (2) 예측하기

In [41]:
# 예측결과를 넣을 df를 생성
# index: userId, columns: total movieId
item_prediction_result_df3 = pd.DataFrame(index= user_movie_matrix.columns, columns=user_movie_matrix.index.values)  # row에 user별로 예측값 넣어주기 위해서, transpose 해서 넣기
item_prediction_result_df3

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [42]:
from tqdm import tqdm
# 2. cur_user의 rating과 , weight 가져오기
check = 0
grouped_rating_df = train_df.groupby('userId')
for user_id, user_ratings_df in tqdm(grouped_rating_df):
    # print()
    # print(user_id)
    # print()
    user_ratings_df = user_ratings_df.sort_values('movieId').set_index('movieId')  # item id로 정렬하고, set_index로 item id 넣어주기 (movieId column은 삭제됨)
    # print(user_ratings_df)

    ## [1] cur_user의 weight 가져오기 : (row : user가 평가한 item, col : 전체 item)
    user_sim_item_df = item_cossim_df.loc[user_ratings_df.index]   # index가 item id인 상황
    # print(user_sim_item_df)

    ## [2] cur_user의 rating 가져오기
    cur_user_rating = user_ratings_df['rating']


    ## [3] cur_user의 전체 item에 대한 weighted_sum
    weighted_sum = cur_user_rating.T.dot(user_sim_item_df)  # (1 x m) dot (m x len(item)) : m은 cur_user가 평가한 item의 개수
    # print(weighted_sum)

    ## [4] weighted_sum을 weight의 절대값 합으로 나눈 것
    # print()
    weight_abs_sum = np.absolute(user_sim_item_df).sum(axis=0)
    pred = weighted_sum/weight_abs_sum
    # pred = weighted_sum/len(user_sim_item_df)
    # print(pred)

    item_prediction_result_df3.loc[user_id] = pred
    # if idcheckx == 0:
    #     break
item_prediction_result_df3

100%|██████████| 610/610 [00:36<00:00, 16.67it/s]


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,129,132,135,137,140,141,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,288,289,290,291,292,293,295,296,...,166946,167018,167036,167064,167296,167370,167380,167538,167570,167732,167746,167772,167790,167854,168026,168090,168144,168218,168248,168250,168252,168254,168266,168326,168350,168366,168418,168456,168492,168608,168632,168712,168846,169034,169180,169670,169912,169958,169982,169984,170355,170357,170401,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170945,170957,170993,171011,171023,171251,171495,171701,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172547,172577,172583,172587,172591,172705,172793,172825,172875,172881,172887,173145,173197,173205,173235,173253,173255,173291,173307,173317,173351,173355,173535,173619,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174681,174727,174737,174909,175197,175199,175293,175303,175387,175401,175431,175435,175475,175485,175569,175577,175585,175661,175693,175707,175743,175781,176101,176371,176413,176415,176419,176423,176579,176601,176621,176751,176805,176935,177185,177285,177593,177615,177763,177765,177939,178061,178111,178323,178613,178615,178827,179053,179119,179133,179135,179211,179401,179427,179709,179749,179815,179817,179819,179953,180031,180095,180231,180263,180265,180297,180497,180777,180985,180987,181065,181139,181413,181659,181719,182293,182297,182299,182639,182715,182749,182823,183011,183197,183199,183295,183301,183317,183611,183635,183897,183911,183959,184015,184053,184245,184253,184349,184471,184641,184791,184931,184997,185029,185031,185135,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,190183,190215,190219,191005,193565,193567,193571,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.400924,4.381924,4.338948,4.473993,4.360636,4.37984,4.318761,4.388002,4.175111,4.333627,4.351173,4.332442,4.431016,4.334279,4.286268,4.390126,4.349024,4.367113,4.365292,4.349911,4.379714,4.361965,4.403123,4.377622,4.370136,4.34982,4.357078,4.392164,4.433291,4.293557,4.357495,4.384203,4.402002,4.363613,4.349193,4.371515,3.590562,4.364411,4.380193,4.356876,4.33163,4.367965,4.365997,4.40322,4.35974,4.402319,4.420112,4.388278,4.320307,4.302476,4.432967,4.386993,4.395866,4.356584,4.33824,4.351992,4.471825,4.244001,4.339823,4.335231,4.331519,4.380748,4.338333,4.382889,4.452817,4.384878,4.387191,4.371516,4.352225,4.320307,4.373426,4.312039,4.315816,4.378823,4.411183,4.389351,4.351503,4.317538,4.366845,4.330083,4.361663,4.311402,4.405566,4.346714,4.436516,4.40629,4.314649,4.354992,4.472277,4.247872,3.956834,4.394962,4.399573,4

# 3. RMSE로 평가하기

In [43]:
def evaluate(test_df, prediction_result_df):
  groups_with_movie_ids = test_df.groupby(by='movieId')
  groups_with_user_ids = test_df.groupby(by='userId')
  intersection_movie_ids = sorted(list(set(list(prediction_result_df.columns)).intersection(set(list(groups_with_movie_ids.indices.keys())))))
  intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

  print(len(intersection_movie_ids))
  print(len(intersection_user_ids))

  compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_movie_ids]
  # compressed_prediction_df

  # test_df에 대해서 RMSE 계산
  grouped = test_df.groupby(by='userId')
  result_df = pd.DataFrame(columns=['rmse'])
  for userId, group in tqdm(grouped):
      if userId in intersection_user_ids:
          pred_ratings = compressed_prediction_df.loc[userId][compressed_prediction_df.loc[userId].index.intersection(list(group['movieId'].values))]
          pred_ratings = pred_ratings.to_frame(name='rating').reset_index().rename(columns={'index':'movieId','rating':'pred_rating'})
          actual_ratings = group[['rating', 'movieId']].rename(columns={'rating':'actual_rating'})

          final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['movieId'])
          final_df = final_df.round(4) # 반올림

          # if not final_df.empty:
          #     rmse = sqrt(mean_squared_error(final_df['rating_actual'], final_df['rating_pred']))
          #     result_df.loc[userId] = rmse
          #     # print(userId, rmse)
    
  return final_df

In [44]:
# version1 - 1
result_df = evaluate(test_df, item_prediction_result_df1)
print()
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

4391
609


100%|██████████| 609/609 [00:05<00:00, 112.01it/s]


RMSE: 3.5705224897427597


In [45]:
# version1 - 2  : 0~5사이로 값 맞춰준 것
result_df = evaluate(test_df, item_prediction_result_df1_2)
print()
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

4391
609


100%|██████████| 609/609 [00:04<00:00, 147.14it/s]


RMSE: 3.171355513169888


In [46]:
# version2 
result_df = evaluate(test_df, item_prediction_result_df2)
result_df = result_df.dropna()
# print(result_df)
print()
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

4391
609


100%|██████████| 609/609 [00:04<00:00, 151.71it/s]



RMSE: 0.820540338975106


In [48]:
# version 3
result_df = evaluate(test_df, item_prediction_result_df3)
print()
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

4391
609


100%|██████████| 609/609 [00:04<00:00, 151.21it/s]


RMSE: 0.8253987765577038
